In [1]:
token = 'vk1.a.4qx3s4mxiIn-PB4EDn_RH7Nb1xJ4qbwZSx_I8G2E05NtKBmh4td5Cq35BMGfzvdvrBJ6rt7F-CBrQigLnYmJIdFrZUaYxE2UqTBO8n9K4SDptsCG2MKabCtAUJbcyxO_XDfvnt2LQxSH2ggKNzclzgIaoOdQ_NzJ3BxFt-dKldzsDhT3aVUST9ZercyDQc3RsofOWJnabPCrH4bGrg367A'

In [6]:
import vk_api
import time
import pandas as pd

MAX_POSTS = 2000


def get_all_comments(token: str, group_id: int):
    """Получает все комментарии к постам сообщества ВК и возвращает DataFrame."""
    vk = vk_api.VkApi(token=token)
    api = vk.get_api()

    # Получаем общее количество постов
    first_response = api.wall.get(owner_id=group_id, count=1)
    total_posts = first_response['count']
    print(f"Всего постов в группе: {total_posts}")

    all_comments = []
    offset = 0

    # Цикл по всем постам (100 за раз)
    while offset < min(total_posts, MAX_POSTS):
        print(f"Обрабатываем посты {offset}–{offset + 100}...")
        try:
            posts = api.wall.get(owner_id=group_id, offset=offset, count=100)['items']
        except Exception as e:
            print(f"Ошибка при получении постов: {e}")
            break

        # Для каждого поста собираем комментарии
        for post in posts:
            post_id = post['id']
            try:
                comments = api.wall.getComments(
                    owner_id=group_id,
                    post_id=post_id,
                    count=100,        # максимум за один запрос
                    sort='asc',       # порядок: старые → новые
                    extended=0
                )['items']
            except Exception as e:
                print(f"Ошибка на посте {post_id}: {e}")
                continue

            for c in comments:
                all_comments.append({
                    "post_id": post_id,
                    "comment_id": c['id'],
                    "from_id": c.get('from_id', ''),
                    "date": c.get('date', ''),
                    "text": c.get('text', '').replace('\n', ' ').strip()
                })

            # Пауза, чтобы не превысить лимит API
            time.sleep(0.4)

        offset += 100
        time.sleep(1)

    print(f"Всего собрано комментариев: {len(all_comments)}")

    df = pd.DataFrame(all_comments)
    return df


group_id = -43596631  # Автомобильные истории

comments_df = get_all_comments(token, group_id)

print(comments_df.head())
print(f"\nВсего строк: {len(comments_df)}")

comments_df.to_csv("vk_comments_raw.csv", index=False)



Всего постов в группе: 68246
Обрабатываем посты 0–100...
Обрабатываем посты 100–200...
Обрабатываем посты 200–300...
Обрабатываем посты 300–400...
Обрабатываем посты 400–500...
Обрабатываем посты 500–600...
Обрабатываем посты 600–700...
Обрабатываем посты 700–800...
Обрабатываем посты 800–900...
Обрабатываем посты 900–1000...
Обрабатываем посты 1000–1100...
Обрабатываем посты 1100–1200...
Обрабатываем посты 1200–1300...
Обрабатываем посты 1300–1400...
Обрабатываем посты 1400–1500...
Обрабатываем посты 1500–1600...
Обрабатываем посты 1600–1700...
Обрабатываем посты 1700–1800...
Обрабатываем посты 1800–1900...
Обрабатываем посты 1900–2000...
Всего собрано комментариев: 3424
   post_id  comment_id    from_id        date  \
0   416545      416546  462341889  1571124019   
1   416545      416547   14483391  1571124180   
2   416545      416552   84044733  1571127500   
3   416545      416605   35314691  1571135870   
4   416545      417070   73348279  1571267523   

                        